1\. Write a function that converts number representation (bin<->dec<->hex)

In [38]:
def convert(number,frombase,tobase): ##FOR EXAMPLE IF I WANT TO CONVERT (29)dec to bin: convert(29,10,2)
    if frombase==2:
        if tobase==10:
            return int(number,2)
        elif tobase==16:
            return hex(int(number,2))
        elif tobase==2:
            return number
        else:
            return 'error'
    elif frombase==10:
        if tobase==2:
            return bin(number)
        elif tobase==16:
            return hex(number)
        elif tobase==10:
            return number
        else:
            return 'error'
    elif frombase==16:
        if tobase==10:
            return int(number,16)
        elif tobase==16:
            return hex(int(number,16))
        elif tobase==16:
            return number
        else:
            return 'error'
    else:
        return 'error'
        
a=39;
print("The number "+str(a)+" will be converted in bin, hex and then reconverted to decimal: ",convert(convert(convert(a,10,2),2,16),16,10))

print(convert(1,2,3)) ##THIS SHOULD RETURN ERROR

The number 39 will be converted in bin, hex and then reconverted to decimal:  39
error


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [5]:
import random

def BinToDec(value):
    try:
        return int(value, 2)
    except ValueError:
        return "Invalid binary Value"

a=""; i=0                         #
while i<=31:                      # Generates a random number of 32 bits
    a=a+str(random.randint(0,1))  #
    i+=1                          #

print(a)

def FP(a):
    n=[int(x) for x in str(a)]
    if n[0]==1:
        sign=-1;
        print("sign: -")
    else:
        sign=+1;
        print("sign: +")
    exp=""
    for i in range(1,9):
        exp=exp+str(n[i])
    print("exponent: "+exp+" converted: "+str(BinToDec(exp)))
    mant=""
    for i in range(9,32):
        mant=mant+str(n[i])
    order=len(str(mant))
    print("mantissa: "+mant+" converted: "+str(BinToDec(mant)))
    
    print("\nSingle precision floating point: "+str(sign*(1+BinToDec(mant)*10**(-order))*2**(BinToDec(exp)-127)))
    
FP(a)

10100111111010111111001001111100
sign: -
exponent: 01001111 converted: 79
mantissa: 11010111111001001111100 converted: 7074428

Single precision floating point: -3.552713678800501e-15


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [2]:
import sys
a=sys.maxsize; b=1;
while True:
    try: 
        a*=2
    except: 
        print(a)
        break;
while True:
    try: 
        b/=2;
    except: 
        print(b)
        break;

...


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [13]:
i=2;j=2;
while i!=i+1/(j+1): #checks if the current value of i is equal to the next, if not continues
    i+=1/(j+1)      #adds to i iteratively 1/(j+1) (next value of i)
    j+=1
print(i)

KeyboardInterrupt: 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [22]:
from math import sqrt;
def sol_a(a,b,c):
    return ((-b+sqrt(b**2-4*a*c))/(2*a) , (-b-sqrt(b**2-4*a*c))/(2*a))
def sol_b(a,b,c):
    return ((-b+sqrt(b**2-4*a*c))/(2*a)*(-b-sqrt(b**2-4*a*c))/(-b-sqrt(b**2-4*a*c)) , 
            (-b-sqrt(b**2-4*a*c))/(2*a)*(-b+sqrt(b**2-4*a*c))/(-b+sqrt(b**2-4*a*c)))
def sol_c(a,b,c):
    print (  float(  hex((-b+sqrt(b**2-4*a*c))/(2*a)*(-b-sqrt(b**2-4*a*c))/(-b-sqrt(b**2-4*a*c))),16  ) , 
             float(  hex((-b-sqrt(b**2-4*a*c))/(2*a)*(-b+sqrt(b**2-4*a*c))/(-b+sqrt(b**2-4*a*c))),16  )  )
    print (  float(  hex((-b+sqrt(b**2-4*a*c))/(2*a) ),16) , float( hex((-b-sqrt(b**2-4*a*c))/(2*a)),16 )  )

a=0.001; b=1000; c=0.001;
print (sol_a(a,b,c))
print (sol_b(a,b,c))
print("They are different because the multiplication term needs a higher number of decimal places and this propagates to the final result. This can be avoided by explicitely controlling the final number of digits:")
sol_c(a,b,c)

(-9.999894245993346e-07, -999999.999999)
(-9.999894245993346e-07, -999999.9999989999)
They are different because the multiplication term needs a higher number of decimal places and this propagates to the final result. This can be avoided by explicitely controlling the final number of digits:


TypeError: 'float' object cannot be interpreted as an integer

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
